In [ ]:
//everyone's physician
//g4+gsl
#pragma cling add_library_path("/usr/local/lib")
#pragma cling load("libgsl.dylib")
#pragma cling load("libG4processes.dylib")
#include <iostream>
#include "G4LegendrePolynomial.hh"
#include <random>
#include <cstdlib>
#include "gsl/gsl_sf_legendre.h"
#include <stdio.h> 
#include <stdlib.h> 
#include <chrono>

{
    G4LegendrePolynomial lp;
    gsl_sf_legendre_t norm = GSL_SF_LEGENDRE_NONE;
    std::random_device rd;
    std::mt19937 mt(rd());
    std::uniform_real_distribution<double> xdist(-1, 1);
    

       for (int l=0; l < 10; ++l){
           double gsl_acc_time_ns = 0;
           double g4_acc_time_ns = 0;
           double sum_diff2 = 0;
           
           std::vector<double> result_array(gsl_sf_legendre_array_n(l), 0);
           
           
           for (int i=0; i < 1000000; ++i){
                double x = xdist(mt);
                
                auto started_gsl = std::chrono::high_resolution_clock::now();
                gsl_sf_legendre_array_e(norm, l, x, -1, &(result_array[0]));
                auto done_gsl = std::chrono::high_resolution_clock::now(); 
                 
                gsl_acc_time_ns += std::chrono::duration_cast<std::chrono::nanoseconds>(done_gsl-started_gsl).count();
                
                for (int m = 0; m <= l; ++m){
                    auto started_g4 = std::chrono::high_resolution_clock::now();
                    double alpg4 = lp.EvalAssocLegendrePoly(l, m, x); 
                    auto done_g4 = std::chrono::high_resolution_clock::now(); 
                    g4_acc_time_ns += std::chrono::duration_cast<std::chrono::nanoseconds>(done_g4-started_g4).count(); 
                    
               
                    double alp_gsl = result_array[gsl_sf_legendre_array_index(l, m)] ;
                    sum_diff2 += pow(alp_gsl - alpg4, 2);
                      
               } // loop over m
           } // loop over i
            std::cout << "L = " << l << ": sum of times for gsl:" <<   gsl_acc_time_ns * 1e-9 << " seconds" << std::endl;
            std::cout << "L = " << l << ": sum of times for g4:" <<   g4_acc_time_ns * 1e-9 << " seconds" << std::endl;
            std::cout << "L = " << l << ": sum of differences squared:" <<   sum_diff2 * 1e-18 << " seconds" << std::endl;
       } // loop over l

}